In [210]:
# imports de bibliotecas
import numpy as np
import mne
from scipy.signal import welch
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()


In [211]:
def print_graphs(X):
    for i in range(X.shape[1]):
        plt.plot(X[:,i])
    plt.title('Domínio do tempo')
    plt.show()

    for i in range(X.shape[1]):
        plt.psd(X[:,i], Fs=250)
    plt.title('Domínio da frequência')
    plt.show()

    for i in range(X.shape[1]):
        plt.specgram(X[i,:], Fs=250)
    plt.title('Espectrograma')
    plt.show()

In [212]:
# definição do info (MNE)

n_channels = 8
ch_types = ['eeg'] * n_channels
sfreq = 250
ch_names = ["F3", "Fz", "F4", "C3", "Cz", "C4", "P3", "P4"]
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
info.set_montage("standard_1020")

Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,11 points
Good channels,8 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,250.00 Hz
Highpass,0.00 Hz
Lowpass,125.00 Hz


In [213]:
# carregamento e organização dos dados de: Maria

ga = [
  '../dataset/maria/ia/OpenBCI-RAW-2023-09-28_16-51-25.txt', # aula: 00:08 - 20:02, teste: 20:23 - 23:10
]
tf = [
  '../dataset/maria/tf/OpenBCI-RAW-2023-11-07_13-17-01.txt', # basal: 3:28, teste: 4:28 - 12:29
]
sr = 250                  # taxa de amostragem

ga_ob = [np.loadtxt(txt, delimiter=',', skiprows=5, usecols=range(1, 9)) for txt in ga]
data_ga = ga_ob[0][8*sr:1202*sr,:]
data_ga_test = ga_ob[0][1223*sr:1390*sr,:]

tf_ob = [np.loadtxt(txt, delimiter=',', skiprows=5, usecols=range(1, 9)) for txt in tf]
data_basal = tf_ob[0][208*sr:268*sr,:]
data_tf = tf_ob[0][268*sr:749*sr,:]

# Insulina Ativa
x_ga = mne.io.RawArray(data_ga.T, info)
# Insulina Ativa Teste
x_ga_test = mne.io.RawArray(data_ga_test.T, info)
# Teste final
x_tf = mne.io.RawArray(data_tf.T, info)
# Basal
x_basal = mne.io.RawArray(data_basal.T, info)

# x_ga.set_eeg_reference(ref_channels='average') # CAR
# x_ga_test.set_eeg_reference(ref_channels='average') # CAR
# x_tf.set_eeg_reference(ref_channels='average') # CAR
# x_basal.set_eeg_reference(ref_channels='average') # CAR
print('x_tf',x_tf.times[-1])
print('x_basal',x_basal.times[-1])

X = {
  'ga': x_ga,
  'ga_test': x_ga_test,
  'tf': x_tf,
}

Creating RawArray with float64 data, n_channels=8, n_times=298500
    Range : 0 ... 298499 =      0.000 ...  1193.996 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=41750
    Range : 0 ... 41749 =      0.000 ...   166.996 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=120250
    Range : 0 ... 120249 =      0.000 ...   480.996 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=15000
    Range : 0 ... 14999 =      0.000 ...    59.996 secs
Ready.
x_tf 480.996
x_basal 59.996


In [214]:
# filtragem de todos os dados

for key in X:
    X[key].notch_filter(freqs=60)
    X[key].filter(l_freq=4, h_freq=100)

x_basal.notch_filter(freqs=60)
x_basal.filter(l_freq=4, h_freq=100)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 1651 samples (6.604 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband e

Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,11 points
Good channels,8 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,250.00 Hz
Highpass,4.00 Hz
Lowpass,100.00 Hz


In [215]:
time = {
  'ga': X['ga'].times[-1],
  'ga_test': X['ga_test'].times[-1],
  'tf': X['tf'].times[-1],
}

sr = 250                  # taxa de amostragem
jump = 5                  # 5s de buffer
size = sr * jump          # quantidade de pontos avalidados
nperseg = 128             # Número de pontos por segmento
noverlap = nperseg // 2   # Quantidade de sobreposição entre segmentos

# Definir os limites das bandas de frequência (em Hz)
theta_band = (4, 8)       # Theta:  4 - 8   Hz
alpha_band = (8, 13)      # Alpha:  8 - 13  Hz
beta_band = (13, 30)      # Beta:  13 - 30  Hz
gamma_band = (30, 100)    # Gamma: 30 - 100 Hz

data_names = ('ga', 'ga_test', 'tf')
# data_names = ('ga')


In [216]:
def calc_great_freq(freqs, psd, buffer):
  results = [0,0,0,0]
  
  avg = np.average(psd, axis=0)

  # Encontrar os índices correspondentes às frequências de interesse
  theta_idxs = np.where((freqs >= theta_band[0]) & (freqs <= theta_band[1]))[0]
  alpha_idxs = np.where((freqs >= alpha_band[0]) & (freqs <= alpha_band[1]))[0]
  beta_idxs =  np.where((freqs >=  beta_band[0]) & (freqs <=  beta_band[1]))[0]
  gamma_idxs = np.where((freqs >= gamma_band[0]) & (freqs <= gamma_band[1]))[0]

  # Calcular a potência em cada banda de frequência
  bands = [
    np.sum(avg[theta_idxs]), 
    np.sum(avg[alpha_idxs]), 
    np.sum(avg[ beta_idxs]), 
    np.sum(avg[gamma_idxs])
  ]
  results[np.argmax(bands)] += 1

  copyBuffer = buffer.copy()
  del copyBuffer[0]
  copyBuffer.append(results)

  acc = np.array([0,0,0,0])
  for col in copyBuffer:
    acc = acc + np.array(col)
  return acc, copyBuffer
  


In [217]:
sample = {
  'focus':[],
  'not_focus':[]
}

buffer = [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]

buffer_size = 5

for k, data in enumerate(X.values()):
  data.set_eeg_reference(ref_channels='average', projection=False)
  for i in range(0, int(time[data_names[k]])-buffer_size-1):
    start_time = i
    stop_time = start_time + buffer_size

    cut = data.get_data(tmin=start_time, tmax=stop_time)
    freqs, psd = welch(cut, fs=sr, nperseg=nperseg, noverlap=noverlap)
    acc, buffer = calc_great_freq(freqs, psd, buffer)
    
    if(acc[2] > 2 or acc[3] > 4):
      sample['focus'].append(scaler.fit_transform(psd))
    else:
      sample['not_focus'].append(scaler.fit_transform(psd))
    
    
# print(sample['focus'])
# print(sample['focus'])

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


In [218]:


cut = x_basal.get_data()
freqs, psd = welch(cut, fs=sr, nperseg=nperseg, noverlap=noverlap)

print(psd.shape)
# print(scaler.fit_transform(psd))
# média das médias de potência de todos os canais para estimar o ruído de fundo
# mean = np.mean(psd)
estimated_background_noise = np.mean(scaler.fit_transform(psd.T))
print(estimated_background_noise)
# print(noise_power) 

(8, 65)
1.5372318802502168e-17


In [219]:
# agora vamos adaptar ambas características 
# aplicando para o nosso sinal de interesse
totalSample = np.array(sample['focus'] + sample['not_focus'])

target_amplitudes_adjusted = totalSample - estimated_background_noise

# subtraindo o ruído de fundo das amplitudes
narrow_band_SNR = 10 * np.log10(target_amplitudes_adjusted / estimated_background_noise)
# print(narrow_band_SNR)
print('narrow_band_SNR',narrow_band_SNR.shape)

total_power = np.sum(target_amplitudes_adjusted)
wide_band_SNR = 10 * np.log10(target_amplitudes_adjusted / total_power)
# print(wide_band_SNR)
print('wide_band_SNR',wide_band_SNR.shape)

narrow_SNR = np.mean(wide_band_SNR, axis=2)
wide_SNR = np.mean(wide_band_SNR, axis=2)

features = np.concatenate((narrow_SNR, wide_SNR), axis=1)
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
features = imputer.fit_transform(features)

print(features.shape)

narrow_band_SNR (1821, 8, 65)
wide_band_SNR (1821, 8, 65)
(1821, 16)


/tmp/ipykernel_6103/70985639.py:8: RuntimeWarning: invalid value encountered in log10
  narrow_band_SNR = 10 * np.log10(target_amplitudes_adjusted / estimated_background_noise)
/tmp/ipykernel_6103/70985639.py:13: RuntimeWarning: invalid value encountered in log10
  wide_band_SNR = 10 * np.log10(target_amplitudes_adjusted / total_power)


In [220]:

divider = len(sample['focus'])

focus = features[:divider]
not_focus = features[divider:]

f_divider = int(divider * 0.7)


focus_train = focus[:f_divider] # 70%
focus_test = focus[f_divider:]  # 30%

nf_divider = int(len(not_focus) * 0.7)
not_focus_train = not_focus[:nf_divider] # 70%
not_focus_test = not_focus[nf_divider:]  # 30%


focus_labels = [0] * f_divider
not_focus_labels = [1] * nf_divider

# train_data = not_focus_train + focus_train 
train_data = np.concatenate((not_focus_train, focus_train), axis=0)
 
train_label = not_focus_labels + focus_labels

train_data # dados 
train_label # labels
train_data = np.array(train_data)
# print(train_data.shape)
# train_data_imputed = imputer.fit_transform(train_data)

clf = svm.SVC(kernel='linear')
# clf.fit(train_data_imputed, train_label)
clf.fit(train_data, train_label)

SVC(kernel='linear')

In [221]:
print("focus_train:",len(focus_train))
print("focus_test:",len(focus_test))
print("not_focus_train:",len(not_focus_train))
print("not_focus_test:",len(not_focus_test))

focus_train: 413
focus_test: 177
not_focus_train: 861
not_focus_test: 370


In [222]:
# print(focus_test.shape)
# focus_test_imputed = imputer.fit_transform(focus_test)
# print(focus_test_imputed.shape)

predict_focus = clf.predict(focus_test)
focus_correct = 0
for data in predict_focus:
  if data == 0:
    focus_correct+=1

# not_focus_test_imputed = imputer.fit_transform(not_focus_test)
not_focus_correct = 0
predict_not_focus = clf.predict(not_focus_test)
for data in predict_not_focus:
  if data == 1:
    not_focus_correct+=1


focus_percent = (focus_correct / len(focus_test))*100
not_focus_percent = (not_focus_correct / len(not_focus_test))*100
total_focus_percent = ((not_focus_correct+focus_correct) / (len(focus_test)+len(not_focus_test)))*100

print(f"FOCUS: {focus_correct}/{len(focus_test)} ({focus_percent}%) correct")
print(f"NOT FOCUS: {not_focus_correct}/{len(not_focus_test)} ({not_focus_percent}%) correct")
print(f"TOTAL: {not_focus_correct+focus_correct}/{len(focus_test)+len(not_focus_test)}({total_focus_percent}%) correct")

FOCUS: 110/177 (62.14689265536724%) correct
NOT FOCUS: 247/370 (66.75675675675676%) correct
TOTAL: 357/547(65.26508226691043%) correct


In [223]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=25)
knn.fit(train_data, train_label)


knn_focus = knn.predict(focus_test)
knn_not_focus = knn.predict(not_focus_test)

knn_focus_correct = 0
for data in knn_focus:
  if data == 0:
    knn_focus_correct+=1

knn_not_focus_correct = 0
for data in knn_not_focus:
  if data == 1:
    knn_not_focus_correct+=1


knn_f_percent = (knn_focus_correct/len(focus_test)) * 100
knn_nf_percent = (knn_not_focus_correct/len(not_focus_test)) * 100
print(f"FOCUS: {knn_focus_correct}/{len(focus_test)} ({knn_f_percent}%) correct")
print(f"NOT FOCUS: {knn_not_focus_correct}/{len(not_focus_test)} ({knn_nf_percent}%)  correct")


FOCUS: 110/177 (62.14689265536724%) correct
NOT FOCUS: 264/370 (71.35135135135135%)  correct
